In [ ]:
# Tengo un problema con los datos, no se corrigieron bien, tengo que unir dfs
# Librerías:
import pandas as pd

import unicodedata
import re


In [40]:
# Cargar datos de los nombres de empresas con correcciones
df_corregido = pd.read_csv("empresas_limpias_corregidas_final.csv")

In [41]:
# Cargar datos del diccionario de scraping
df_scraping = pd.read_csv("diccionario_nombres_corregidos.csv")

In [42]:
# Unir los dataframes por la columna 'NOMBRE_LIMPIO' y 'NOMBRE_ORIGINAL'
df_merged = df_corregido.merge(
    df_scraping,
    left_on="ADJUDICATARIO",
    right_on="NOMBRE_ORIGINAL",
    how="left",
    suffixes=("", "_MANUAL")
)


In [43]:
df_merged

,Nº,ADJUDICATARIO,CIF,LICITACIONES ADJUDICADAS,NOMBRE_LIMPIO,NOMBRE_CORREGIDO,MATCH_SCORE,NOMBRE_CORREGIDO_FINAL,STATUS_CORRECCIÓN,NOMBRE_ORIGINAL,NOMBRE_CORREGIDO_FINAL_MANUAL
0,1,"ACCIONA AGUA, S.A.U.",A95113361,60,"ACCIONA AGUA, S.A.U.","ACCIONA AGUA, S.A.U.",100.0,ACCIONA AGUA SAU,Corregido manual,"ACCIONA AGUA, S.A.U.",ACCIONA AGUA SAU
1,2,"Aceinsa Movilidad, S.A",A84408954,207,"ACEINSA MOVILIDAD, S.A","ACEINSA MOVILIDAD, S.A",100.0,NaN,Corregido manual,"Aceinsa Movilidad, S.A",ACEINSA MOVILIDAD SA
2,3,Agrupacion Europea de Industrias de Transforma...,B83037606,5,AGRUPACION EUROPEA DE INDUSTRIAS DE TRANSFORMA...,AGRUPACION EUROPEA DE INDUSTRIAS DE TRANSFORMA...,100.0,NaN,Corregido manual,Agrupacion Europea de Industrias de Transforma...,AGRUPACION EUROPEA DE INDUSTRIAS DE TRANSFORMA...
3,4,AIJONMUSIC S.A.,A83299750,23,AIJONMUSIC S.A.,AIJONMUSIC S.A.,100.0,AIJONMUSIC SA,Corregido manual,AIJONMUSIC S.A.,AIJONMUSIC SA
4,5,ALBINI MAQUINARIA DE JARDINERIA SLU,B82205030,9,ALBINI MAQUINARIA DE JARDINERIA SLU,ALBINI MAQUINARIA DE JARDINERIA SLU,100.0,ALBINI MAQUINARIA DE JARDINERIA SLU,Sin cambio,ALBINI MAQUINARIA DE JARDINERIA SLU,ALBINI MAQUINARIA DE JARDINERIA SLU
...,...,...,...,...,...,...,...,...,...,...,...
95,96,UTE RETRASOL,287768,1,UTE RETRASOL,UTE RETRASOL,100.0,UTE RETRASOL,Sin cambio,UTE RETRASOL,UTE RETRASOL
96,97,"UTE ROMA INGENIEROS CONSULTORES, S.L.-INCRESCE...",U72559479,1,"UTE ROMA INGENIEROS CONSULTORES, S.L.-INCRESCE...","UTE ROMA INGENIEROS CONSULTORES, S.L.-INCRESCE...",100.0,UTE ROMA INGENIEROS CONSULTORES SL INCRESCENDO...,Corregido manual,"UTE ROMA INGENIEROS CONSULTORES, S.L.-INCRESCE...",UTE ROMA INGENIEROS CONSULTORES SL INCRESCENDO...
97,98,VALORIZA SERVICIOS MEDIOAMBIENTALES S.A,A28760692,207,VALORIZA SERVICIOS MEDIOAMBIENTALES S.A,VALORIZA SERVICIOS MEDIOAMBIENTALES S.A,100.0,VALORIZA SERVICIOS MEDIOAMBIENTALES SA,Corregido manual,VALORIZA SERVICIOS MEDIOAMBIENTALES S.A,VALORIZA SERVICIOS MEDIOAMBIENTALES SA
98,99,"Vistalegre Solutions, S. L.",B14404065,218,"VISTALEGRE SOLUTIONS, S. L.","VISTALEGRE SOLUTIONS, S. L.",100.0,NaN,Corregido manual,"Vistalegre Solutions, S. L.",VISTALEGRE SOLUTIONS SL


In [44]:
# Coicidencias de nombres corregidos final (bien)
print("Registros con match manual:", df_merged["NOMBRE_CORREGIDO_FINAL_MANUAL"].notna().sum())
print("Registros sin match manual:", df_merged["NOMBRE_CORREGIDO_FINAL_MANUAL"].isna().sum())


Registros con match manual: 92
Registros sin match manual: 8


In [45]:
# Rellenar condicionalmente NOMBRE_CORREGIDO_FINAL_MANUAL si está vacío
df_merged["NOMBRE_CORREGIDO_FINAL_MANUAL"] = df_merged.apply(
    lambda row: row["NOMBRE_CORREGIDO"] if pd.isna(row["NOMBRE_CORREGIDO_FINAL_MANUAL"]) else row["NOMBRE_CORREGIDO_FINAL_MANUAL"],
    axis=1
)

# Registrar el origen de la corrección
df_merged["ORIGEN_CORRECCIÓN"] = df_merged.apply(
    lambda row: "AUTOMÁTICA" if row["NOMBRE_CORREGIDO_FINAL_MANUAL"] == row["NOMBRE_CORREGIDO"] else "MANUAL",
    axis=1
)

# Auditoría rápida
print("✔ Registros con corrección manual:", (df_merged["ORIGEN_CORRECCIÓN"] == "MANUAL").sum())
print("✔ Registros con corrección automática:", (df_merged["ORIGEN_CORRECCIÓN"] == "AUTOMÁTICA").sum())


✔ Registros con corrección manual: 72
✔ Registros con corrección automática: 28


In [46]:
df_merged

,Nº,ADJUDICATARIO,CIF,LICITACIONES ADJUDICADAS,NOMBRE_LIMPIO,NOMBRE_CORREGIDO,MATCH_SCORE,NOMBRE_CORREGIDO_FINAL,STATUS_CORRECCIÓN,NOMBRE_ORIGINAL,NOMBRE_CORREGIDO_FINAL_MANUAL,ORIGEN_CORRECCIÓN
0,1,"ACCIONA AGUA, S.A.U.",A95113361,60,"ACCIONA AGUA, S.A.U.","ACCIONA AGUA, S.A.U.",100.0,ACCIONA AGUA SAU,Corregido manual,"ACCIONA AGUA, S.A.U.",ACCIONA AGUA SAU,MANUAL
1,2,"Aceinsa Movilidad, S.A",A84408954,207,"ACEINSA MOVILIDAD, S.A","ACEINSA MOVILIDAD, S.A",100.0,NaN,Corregido manual,"Aceinsa Movilidad, S.A",ACEINSA MOVILIDAD SA,MANUAL
2,3,Agrupacion Europea de Industrias de Transforma...,B83037606,5,AGRUPACION EUROPEA DE INDUSTRIAS DE TRANSFORMA...,AGRUPACION EUROPEA DE INDUSTRIAS DE TRANSFORMA...,100.0,NaN,Corregido manual,Agrupacion Europea de Industrias de Transforma...,AGRUPACION EUROPEA DE INDUSTRIAS DE TRANSFORMA...,MANUAL
3,4,AIJONMUSIC S.A.,A83299750,23,AIJONMUSIC S.A.,AIJONMUSIC S.A.,100.0,AIJONMUSIC SA,Corregido manual,AIJONMUSIC S.A.,AIJONMUSIC SA,MANUAL
4,5,ALBINI MAQUINARIA DE JARDINERIA SLU,B82205030,9,ALBINI MAQUINARIA DE JARDINERIA SLU,ALBINI MAQUINARIA DE JARDINERIA SLU,100.0,ALBINI MAQUINARIA DE JARDINERIA SLU,Sin cambio,ALBINI MAQUINARIA DE JARDINERIA SLU,ALBINI MAQUINARIA DE JARDINERIA SLU,AUTOMÁTICA
...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,UTE RETRASOL,287768,1,UTE RETRASOL,UTE RETRASOL,100.0,UTE RETRASOL,Sin cambio,UTE RETRASOL,UTE RETRASOL,AUTOMÁTICA
96,97,"UTE ROMA INGENIEROS CONSULTORES, S.L.-INCRESCE...",U72559479,1,"UTE ROMA INGENIEROS CONSULTORES, S.L.-INCRESCE...","UTE ROMA INGENIEROS CONSULTORES, S.L.-INCRESCE...",100.0,UTE ROMA INGENIEROS CONSULTORES SL INCRESCENDO...,Corregido manual,"UTE ROMA INGENIEROS CONSULTORES, S.L.-INCRESCE...",UTE ROMA INGENIEROS CONSULTORES SL INCRESCENDO...,MANUAL
97,98,VALORIZA SERVICIOS MEDIOAMBIENTALES S.A,A28760692,207,VALORIZA SERVICIOS MEDIOAMBIENTALES S.A,VALORIZA SERVICIOS MEDIOAMBIENTALES S.A,100.0,VALORIZA SERVICIOS MEDIOAMBIENTALES SA,Corregido manual,VALORIZA SERVICIOS MEDIOAMBIENTALES S.A,VALORIZA SERVICIOS MEDIOAMBIENTALES SA,MANUAL
98,99,"Vistalegre Solutions, S. L.",B14404065,218,"VISTALEGRE SOLUTIONS, S. L.","VISTALEGRE SOLUTIONS, S. L.",100.0,NaN,Corregido manual,"Vistalegre Solutions, S. L.",VISTALEGRE SOLUTIONS SL,MANUAL


In [47]:
# Función de 2da normalización
def normalizar_nombre(nombre):
    if pd.isna(nombre):
        return ""
    
    # Convertir a minúsculas
    nombre = nombre.upper()
    
    # Eliminar tildes y diacríticos (mantiene la ñ)
    nombre = unicodedata.normalize("NFKD", nombre).encode("ASCII", "ignore").decode("utf-8")
    
    # Eliminar signos de puntuación y caracteres especiales
    nombre = re.sub(r"[^\w\s]", " ", nombre)
    
    # Reemplazar múltiples espacios por uno solo
    nombre = re.sub(r"\s+", " ", nombre).strip()
    
    return nombre


In [48]:
df_merged

,Nº,ADJUDICATARIO,CIF,LICITACIONES ADJUDICADAS,NOMBRE_LIMPIO,NOMBRE_CORREGIDO,MATCH_SCORE,NOMBRE_CORREGIDO_FINAL,STATUS_CORRECCIÓN,NOMBRE_ORIGINAL,NOMBRE_CORREGIDO_FINAL_MANUAL,ORIGEN_CORRECCIÓN
0,1,"ACCIONA AGUA, S.A.U.",A95113361,60,"ACCIONA AGUA, S.A.U.","ACCIONA AGUA, S.A.U.",100.0,ACCIONA AGUA SAU,Corregido manual,"ACCIONA AGUA, S.A.U.",ACCIONA AGUA SAU,MANUAL
1,2,"Aceinsa Movilidad, S.A",A84408954,207,"ACEINSA MOVILIDAD, S.A","ACEINSA MOVILIDAD, S.A",100.0,NaN,Corregido manual,"Aceinsa Movilidad, S.A",ACEINSA MOVILIDAD SA,MANUAL
2,3,Agrupacion Europea de Industrias de Transforma...,B83037606,5,AGRUPACION EUROPEA DE INDUSTRIAS DE TRANSFORMA...,AGRUPACION EUROPEA DE INDUSTRIAS DE TRANSFORMA...,100.0,NaN,Corregido manual,Agrupacion Europea de Industrias de Transforma...,AGRUPACION EUROPEA DE INDUSTRIAS DE TRANSFORMA...,MANUAL
3,4,AIJONMUSIC S.A.,A83299750,23,AIJONMUSIC S.A.,AIJONMUSIC S.A.,100.0,AIJONMUSIC SA,Corregido manual,AIJONMUSIC S.A.,AIJONMUSIC SA,MANUAL
4,5,ALBINI MAQUINARIA DE JARDINERIA SLU,B82205030,9,ALBINI MAQUINARIA DE JARDINERIA SLU,ALBINI MAQUINARIA DE JARDINERIA SLU,100.0,ALBINI MAQUINARIA DE JARDINERIA SLU,Sin cambio,ALBINI MAQUINARIA DE JARDINERIA SLU,ALBINI MAQUINARIA DE JARDINERIA SLU,AUTOMÁTICA
...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,UTE RETRASOL,287768,1,UTE RETRASOL,UTE RETRASOL,100.0,UTE RETRASOL,Sin cambio,UTE RETRASOL,UTE RETRASOL,AUTOMÁTICA
96,97,"UTE ROMA INGENIEROS CONSULTORES, S.L.-INCRESCE...",U72559479,1,"UTE ROMA INGENIEROS CONSULTORES, S.L.-INCRESCE...","UTE ROMA INGENIEROS CONSULTORES, S.L.-INCRESCE...",100.0,UTE ROMA INGENIEROS CONSULTORES SL INCRESCENDO...,Corregido manual,"UTE ROMA INGENIEROS CONSULTORES, S.L.-INCRESCE...",UTE ROMA INGENIEROS CONSULTORES SL INCRESCENDO...,MANUAL
97,98,VALORIZA SERVICIOS MEDIOAMBIENTALES S.A,A28760692,207,VALORIZA SERVICIOS MEDIOAMBIENTALES S.A,VALORIZA SERVICIOS MEDIOAMBIENTALES S.A,100.0,VALORIZA SERVICIOS MEDIOAMBIENTALES SA,Corregido manual,VALORIZA SERVICIOS MEDIOAMBIENTALES S.A,VALORIZA SERVICIOS MEDIOAMBIENTALES SA,MANUAL
98,99,"Vistalegre Solutions, S. L.",B14404065,218,"VISTALEGRE SOLUTIONS, S. L.","VISTALEGRE SOLUTIONS, S. L.",100.0,NaN,Corregido manual,"Vistalegre Solutions, S. L.",VISTALEGRE SOLUTIONS SL,MANUAL


In [49]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Nº                             100 non-null    int64  
 1   ADJUDICATARIO                  100 non-null    object 
 2   CIF                            100 non-null    object 
 3   LICITACIONES ADJUDICADAS       100 non-null    int64  
 4   NOMBRE_LIMPIO                  100 non-null    object 
 5   NOMBRE_CORREGIDO               100 non-null    object 
 6   MATCH_SCORE                    100 non-null    float64
 7   NOMBRE_CORREGIDO_FINAL         75 non-null     object 
 8   STATUS_CORRECCIÓN              100 non-null    object 
 9   NOMBRE_ORIGINAL                92 non-null     object 
 10  NOMBRE_CORREGIDO_FINAL_MANUAL  100 non-null    object 
 11  ORIGEN_CORRECCIÓN              100 non-null    object 
dtypes: float64(1), int64(2), object(9)
memory usage: 9.

In [50]:
# Lista de columnas que se quiere conservar
columnas_seleccionadas = ["Nº", "ADJUDICATARIO", "CIF", "NOMBRE_CORREGIDO_FINAL_MANUAL"]

# Crear nuevo DataFrame con esas columnas
df_reducido = df_merged[columnas_seleccionadas].copy()


In [51]:
df_reducido

,Nº,ADJUDICATARIO,CIF,NOMBRE_CORREGIDO_FINAL_MANUAL
0,1,"ACCIONA AGUA, S.A.U.",A95113361,ACCIONA AGUA SAU
1,2,"Aceinsa Movilidad, S.A",A84408954,ACEINSA MOVILIDAD SA
2,3,Agrupacion Europea de Industrias de Transforma...,B83037606,AGRUPACION EUROPEA DE INDUSTRIAS DE TRANSFORMA...
3,4,AIJONMUSIC S.A.,A83299750,AIJONMUSIC SA
4,5,ALBINI MAQUINARIA DE JARDINERIA SLU,B82205030,ALBINI MAQUINARIA DE JARDINERIA SLU
...,...,...,...,...
95,96,UTE RETRASOL,287768,UTE RETRASOL
96,97,"UTE ROMA INGENIEROS CONSULTORES, S.L.-INCRESCE...",U72559479,UTE ROMA INGENIEROS CONSULTORES SL INCRESCENDO...
97,98,VALORIZA SERVICIOS MEDIOAMBIENTALES S.A,A28760692,VALORIZA SERVICIOS MEDIOAMBIENTALES SA
98,99,"Vistalegre Solutions, S. L.",B14404065,VISTALEGRE SOLUTIONS SL


In [52]:
# Función de normalización que convierte a mayúsculas y reemplaza Ñ por N
def normalizar_nombre_mayusculas(nombre):
    if pd.isna(nombre):
        return ""
    
    # Convertir a string y mayúsculas
    nombre = str(nombre).upper()
    
    # Reemplazar Ñ por N explícitamente
    nombre = nombre.replace("Ñ", "N")
    
    # Eliminar tildes y diacríticos
    nombre = unicodedata.normalize("NFKD", nombre).encode("ASCII", "ignore").decode("utf-8")
    
    # Eliminar puntos y signos de puntuación
    nombre = re.sub(r"[^\w\s]", " ", nombre)
    
    # Reemplazar múltiples espacios por uno solo
    nombre = re.sub(r"\s+", " ", nombre).strip()
    
    return nombre


In [ ]:
# Aplicar la normalización a la columna NOMBRE_CORREGIDO_FINAL_MANUAL
df_reducido["NOMBRE_CORREGIDO_FINAL_MANUAL_NORMALIZADO"] = df_reducido["NOMBRE_CORREGIDO_FINAL_MANUAL"].apply(normalizar_nombre_mayusculas)

In [55]:
# Verificar resultados
df_reducido

,Nº,ADJUDICATARIO,CIF,NOMBRE_CORREGIDO_FINAL_MANUAL,NOMBRE_CORREGIDO_FINAL_MANUAL_NORMALIZADO
0,1,"ACCIONA AGUA, S.A.U.",A95113361,ACCIONA AGUA SAU,ACCIONA AGUA SAU
1,2,"Aceinsa Movilidad, S.A",A84408954,ACEINSA MOVILIDAD SA,ACEINSA MOVILIDAD SA
2,3,Agrupacion Europea de Industrias de Transforma...,B83037606,AGRUPACION EUROPEA DE INDUSTRIAS DE TRANSFORMA...,AGRUPACION EUROPEA DE INDUSTRIAS DE TRANSFORMA...
3,4,AIJONMUSIC S.A.,A83299750,AIJONMUSIC SA,AIJONMUSIC SA
4,5,ALBINI MAQUINARIA DE JARDINERIA SLU,B82205030,ALBINI MAQUINARIA DE JARDINERIA SLU,ALBINI MAQUINARIA DE JARDINERIA SLU
...,...,...,...,...,...
95,96,UTE RETRASOL,287768,UTE RETRASOL,UTE RETRASOL
96,97,"UTE ROMA INGENIEROS CONSULTORES, S.L.-INCRESCE...",U72559479,UTE ROMA INGENIEROS CONSULTORES SL INCRESCENDO...,UTE ROMA INGENIEROS CONSULTORES SL INCRESCENDO...
97,98,VALORIZA SERVICIOS MEDIOAMBIENTALES S.A,A28760692,VALORIZA SERVICIOS MEDIOAMBIENTALES SA,VALORIZA SERVICIOS MEDIOAMBIENTALES SA
98,99,"Vistalegre Solutions, S. L.",B14404065,VISTALEGRE SOLUTIONS SL,VISTALEGRE SOLUTIONS SL


In [58]:
# Seleccionar solo las columnas deseadas
columnas_exportar = ["Nº", "NOMBRE_CORREGIDO_FINAL_MANUAL_NORMALIZADO", "CIF"]
df_exportado = df_reducido[columnas_exportar].copy()

# Guardar como CSV
df_exportado.to_csv("nombres_normalizados_para_scraping.csv", index=False, encoding="utf-8-sig")
